## filmmakers 게시글 제목, 파일명 다운 (실행시키지 말 것!!)

In [ ]:
dict = {}

import requests
from bs4 import BeautifulSoup
import time
import numpy as np

for page in range(1,22):
    url = f"https://www.filmmakers.co.kr/koreanScreenplays/page/{page}"

    resp = requests.get(url)

    soup= BeautifulSoup(resp.text, 'html5lib')
    sel = "#board-list > table > tbody > tr > td:nth-child(1) > a"
    titles = soup.select(sel)
    
    for tag in titles:
        sub_url = "https://www.filmmakers.co.kr" + tag['href']
        resp = requests.get(sub_url)

        soup= BeautifulSoup(resp.text, 'html5lib')

        sel = "#board-content > div:nth-child(2) > div > div > p > a"
        links = soup.select(sel)
        for item in links:
            file_url = "https://www.filmmakers.co.kr/" + item['href'][1:]
            print(item.text)
            
            with open(item.text.strip(),"wb") as f:
                f.write(resp.content)
            time.sleep(2)
            
            dict[tag.text[:-3].strip()] = item.text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive


###  --------------------------------
### 게시글 제목, 파일명 따로 추출(여기서부터 실행)

In [ ]:
title = []
file_names =[]

import requests
from bs4 import BeautifulSoup
import time
import numpy as np
from tqdm import tqdm

for page in range(1,22):
    url = f"https://www.filmmakers.co.kr/koreanScreenplays/page/{page}"

    resp = requests.get(url)

    soup= BeautifulSoup(resp.text, 'html5lib')
    sel = "#board-list > table > tbody > tr > td:nth-of-type(1) > a"
    titles = soup.select(sel)
    
    for tag in tqdm(titles):
        sub_url = "https://www.filmmakers.co.kr" + tag['href']
        resp = requests.get(sub_url)

        soup= BeautifulSoup(resp.text, 'html5lib')

        sel = "#board-content > div:nth-of-type(2) > div > div > p > a"
        links = soup.select(sel)
            
        for item in links:
            # print(item.text)
            title.append(tag.text[:-2].strip())
            file_names.append(item.text[1:])


100%|██████████| 14/14 [00:14<00:00,  1.06s/it]


In [ ]:
len(title)

409

In [ ]:
len(file_names)

409

In [ ]:
title

### 게시글 제목에서 영화명 추출

In [ ]:
import re 
edit_title=[]
for k in title:
    k = re.sub('시나리오|입니다|제본고|각본|[a-zA-Z]|올립니다|최종고|[\([가-힇]+\)]','',k)
    if '<' in k:
        k = re.split('[<>]', k)[1]
    elif '[' in k:
        k = re.split('[\[\]]', k)[1]
    elif '"' in k:
        k = re.split('[\"\"]', k)[1]
    edit_title.append(k)

### 제목/파일명 df

In [ ]:
import pandas as pd
t_series = pd.Series(edit_title)
f_series = pd.Series(file_names)
name_df = pd.concat([t_series, f_series], axis=1)
name_df

,0,1
0,파수꾼,파수꾼.hwp
1,변산,%BA%AF%BB%EA.pdf
2,어린 의뢰인 오리지널,어린의뢰인_원작각본(배포용).hwp
3,교회누나,교회누나-최종고2(제본용).hwp
4,대관람차,시나리오합본_161001_우주.pdf
...,...,...
404,8월의 크리스마스,8월의_크리스마스.hwp
405,박하사탕,candy2_6.hwp
406,시월애,swe.zip
407,파란대문,blue.txt


In [ ]:
name_df.columns = ['영화명', '파일명']
name_df

,영화명,파일명
0,파수꾼,파수꾼.hwp
1,변산,%BA%AF%BB%EA.pdf
2,어린 의뢰인 오리지널,어린의뢰인_원작각본(배포용).hwp
3,교회누나,교회누나-최종고2(제본용).hwp
4,대관람차,시나리오합본_161001_우주.pdf
...,...,...
404,8월의 크리스마스,8월의_크리스마스.hwp
405,박하사탕,candy2_6.hwp
406,시월애,swe.zip
407,파란대문,blue.txt


### 중복 파일 제거 

In [ ]:
for k in file_names:
    if file_names.count(k) != 1:
        print(k)

지구를 지켜라.hwp
추격자.hwp
타짜.hwp
날보러와요_김광림작.hwp
날보러와요_김광림작.hwp
오버더레인보우.hwp
사랑.doc
작전.pdf
작전.pdf
조용한세상.hwp
달콤한 인생.hwp
세븐데이즈.hwp
추격자.hwp
세븐데이즈.hwp
사랑.doc
천하장사마돈나.hwp
지구를 지켜라.hwp
조용한세상.hwp
라디오스타.hwp
타짜.hwp
라디오스타.hwp
천하장사마돈나.hwp
구타유발자들.pdf
달콤한 인생.hwp
구타유발자들.pdf
효자동.hwp
효자동.hwp
올드보이.hwp
올드보이.hwp
오버더레인보우.hwp


In [ ]:
name_df= name_df.drop_duplicates(['파일명'], keep='first', ignore_index=True)

In [ ]:
name_df

,영화명,파일명
0,파수꾼,파수꾼.hwp
1,변산,%BA%AF%BB%EA.pdf
2,어린 의뢰인 오리지널,어린의뢰인_원작각본(배포용).hwp
3,교회누나,교회누나-최종고2(제본용).hwp
4,대관람차,시나리오합본_161001_우주.pdf
...,...,...
389,8월의 크리스마스,8월의_크리스마스.hwp
390,박하사탕,candy2_6.hwp
391,시월애,swe.zip
392,파란대문,blue.txt


In [ ]:
file_names_unique = set(file_names)
len(file_names_unique)

394

## 영화 코드, 장르, 배역명, 관람 등급, 줄거리 크롤링 

In [ ]:
from bs4 import BeautifulSoup
import numpy as np 
import re 

In [ ]:
url = "https://movie.naver.com/movie/bi/mi/basic.naver?code=198434"
# url = "https://movie.naver.com/movie/search/result.naver?query=변산&section=all&ie=utf8"
res = requests.get(url)

In [ ]:
soup=BeautifulSoup(res.text,'lxml')

In [ ]:
# 링크 찾는 크롤링
# list(map(lambda x: x.find('a')['href'], soup.find("ul", {"class": "search_list_1"}).find_all('li')))

['/movie/bi/mi/basic.naver?code=164151']

In [ ]:
# 내용 찾는 크롤링
genre = [soup.find("dl", {"class":"info_spec"}).find_all('dd')[0].find('span').text.strip("\n")]
actors = soup.find("dl", {"class":"info_spec"}).find_all('dd')[2].find('p').text

print(genre, actors)

['코미디'] 제이콥 코헤인, 케이티 허버드


In [ ]:
p = re.compile(r"\([가-힇]+\)")

def infoc(movie):
    print(movie, ":")
    url = f'https://movie.naver.com/movie/search/result.naver?query={movie}&section=all&ie=utf8'
    res = requests.get(url)
    soup=BeautifulSoup(res.text,'lxml')
    if res.status_code == 200:
        soup=BeautifulSoup(res.text,'lxml')
        # sel = '#old_content > ul:nth-of-type(4) > li:nth-of-type(1) > dl > dt > a'
        # link = soup.select(sel)
        try:
          link = list(map(lambda x: x.find('a')['href'], soup.find("ul", {"class": "search_list_1"}).find_all('li')))
          if len(link) != 0:
              # code = link[0]['href'][-5:]
              code = link[0].split("=")[-1]
              print(code,":")
              url = f'https://movie.naver.com/movie/bi/mi/basic.naver?code={code}'
              res = requests.get(url)
              soup=BeautifulSoup(res.text,'lxml')
              try:
                  genre = re.sub('\s', '', soup.find("dl", {"class":"info_spec"}).find_all('dd')[0].find('span').text.strip("\n"))
                  actors = soup.find("dl", {"class":"info_spec"}).find_all('dd')[2].find('p').text
                  result = genre, actors
                  print(result)
                  return result
              except:
                  print("soup problem")
                  return [np.nan,np.nan]      
          else:
              print("Link does not exit")
              return [np.nan,np.nan]
        except:
          print("Search Result is 0")
          return [np.nan,np.nan]

In [ ]:
info_dict = {}
for names in name_df['영화명']:
    info_dict[names] = infoc(names)

In [ ]:
import copy
info_dict_df = copy.deepcopy(info_dict)

In [ ]:
info_dict_df[list(info_dict_df.keys())[10]][0]

['코미디']

In [ ]:
for k, v in info_dict_df.items():
  genre, actors = v
  if type(genre) == list:
    genre = str(genre[0])
  elif type(genre) == float:
    genre = "NaN"
  else:
    genre = genre.split(",")[0]

  actors = str(actors)
  actors_r = p.findall(actors)
  if len(actors_r) == 0:
    actors_r = actors
  else:
    actors_r = str(actors_r)
    actors_r = re.sub("\W", " ", actors_r)

  info_dict_df[k] = [genre, actors_r]

In [ ]:
info_dict_df

In [ ]:
import pickle

with open('info_dict_df_poc.pkl', 'wb') as f:
  pickle.dump(info_dict_df, f)

In [ ]:
import pandas as pd
# info_df = pd.DataFrame.from_dict(info_dict, orient='index', columns = ['장르', '배역', '관람등급', '줄거리'])
info_df = pd.DataFrame.from_dict(info_dict_df, orient='index', columns = ['장르', '배역'])
info_df['영화명'] = info_df.index
info_df.reset_index(drop=True)

,장르,배역,영화명
0,드라마,기태 동윤 희준,파수꾼
1,드라마,학수 선미,변산
2,NaN,nan,어린 의뢰인 오리지널
3,멜로/로맨스,유아희 양백지,교회누나
4,드라마,우주 하루나 스노우,대관람차
...,...,...,...
377,NaN,nan,8월의 크리스마스
378,NaN,nan,박하사탕
379,드라마,성현 은주,시월애
380,NaN,nan,파란대문


In [ ]:
movie_df = pd.merge(name_df,info_df, on='영화명', how='inner')
movie_df

,영화명,파일명,장르,배역
0,파수꾼,파수꾼.hwp,드라마,기태 동윤 희준
1,변산,%BA%AF%BB%EA.pdf,드라마,학수 선미
2,어린 의뢰인 오리지널,어린의뢰인_원작각본(배포용).hwp,NaN,nan
3,교회누나,교회누나-최종고2(제본용).hwp,멜로/로맨스,유아희 양백지
4,교회누나,녹음대본_교회누나1.xls,멜로/로맨스,유아희 양백지
...,...,...,...,...
389,8월의 크리스마스,8월의_크리스마스.hwp,NaN,nan
390,박하사탕,candy2_6.hwp,NaN,nan
391,시월애,swe.zip,드라마,성현 은주
392,파란대문,blue.txt,NaN,nan


In [ ]:
movie_df.to_csv("movie_df.csv", encoding="utf-8")

In [ ]:
for k in range(0,394):
    if ((len(movie_df.iloc[k,:]['장르'])==0)|(len(movie_df.iloc[k,:]['배역'])==0)) ==True:
        movie_df.drop(index=k)
movie_df

,영화명,파일명,장르,배역
0,파수꾼,파수꾼.hwp,[드라마],기태 동윤 희준
1,변산,%BA%AF%BB%EA.pdf,[드라마],학수 선미
2,어린 의뢰인 오리지널,어린의뢰인_원작각본(배포용).hwp,[NaN],nan
3,교회누나,교회누나-최종고2(제본용).hwp,[멜로/로맨스],유아희 양백지
4,교회누나,녹음대본_교회누나1.xls,[멜로/로맨스],유아희 양백지
...,...,...,...,...
389,8월의 크리스마스,8월의_크리스마스.hwp,[NaN],nan
390,박하사탕,candy2_6.hwp,[NaN],nan
391,시월애,swe.zip,[드라마],성현 은주
392,파란대문,blue.txt,[NaN],nan


In [ ]:
for k in range(0,394):
    print((len(movie_df.iloc[k,:]['장르'])==0)|(len(movie_df.iloc[k,:]['배역'])==0))

In [ ]:
for i in range(len(movie_df)):
  print(len(movie_df.loc[i, "장르"]))

## hwp to txt 코드

In [ ]:
import os
from time import sleep

path = "C:\\Users\\elina\\Desktop\\DSL\\Untitled Folder\\"
exefile = 'hwp5txt'
res = []
for root, dirs, files in os.walk(path):
    rootpath =os.path.join(os.path.abspath(path), root)
    for file in files:
        if file[-3:] == 'hwp':
            filepath = os.path.join(rootpath, file)
            res.append(filepath) 

    for result in res:
        filename = result[:-4]+ ".txt"
        output = '--output ' + '"' + filename + '"'
        result = '"' + result + '"'
        print(exefile + " " + output + " " + result)
        os.system(exefile + " " + output + " " + result)
        sleep(0.5)

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\070320_뜨거운것이_좋아.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\070320_뜨거운것이_좋아.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\0920시나리오-제본용.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\0920시나리오-제본용.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\3807_우아한거짓말.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\3807_우아한거짓말.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\7[1].1고_쿨.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\7[1].1고_쿨.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\7급공무원.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\7급공무원.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\8월의_크리스마스.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\8월의_크리스마스.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\bunji.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\bunji.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Unti

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\날보러와요.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\날보러와요.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\날보러와요_김광림작.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\날보러와요_김광림작.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\남극일기.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\남극일기.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\남자이야기.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\남자이야기.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\내 심장을 쏴라.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\내 심장을 쏴라.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\내_깡패같은_애인.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\내_깡패같은_애인.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\내머리속의지우개.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\내머리속의지우개.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\너는 내 운명.txt" "C:\Users

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\범죄.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\범죄.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\베사3.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\베사3.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\베스트셀러.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\베스트셀러.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\복수는_나의것_시나리오.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\복수는_나의것_시나리오.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\봄날은_간다_5고_(2월21일).txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\봄날은_간다_5고_(2월21일).hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\봉준호 '지리멸렬'.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\봉준호 '지리멸렬'.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\부당거래.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\부당거래.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\부산행.txt" "C:\U

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\역도산.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\역도산.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\역전의_명수1.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\역전의_명수1.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\연애소설(97).txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\연애소설(97).hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\연애의 온도.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\연애의 온도.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\연애의_목적.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\연애의_목적.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\열혈남아.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\열혈남아.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\엽기적인_그녀.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\엽기적인_그녀.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\영어완전정복.txt" "C:\Users\elina\Desktop\DS

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\청연(靑燕).txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\청연(靑燕).hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\청춘만화.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\청춘만화.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\첼로-홍미주일가 살인사건.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\첼로-홍미주일가 살인사건.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\초록 물고기.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\초록 물고기.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\초록.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\초록.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\최강로맨스.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\최강로맨스.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\추격자.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\추격자.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\친구.txt" "C:\Users\elina\Desktop\DSL\Untitl

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\single.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\single.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\unknown.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\unknown.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\weryungje.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\weryungje.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\ymca야구단.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\ymca야구단.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\[시나리오] 박수칠 때 떠나라.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\[시나리오] 박수칠 때 떠나라.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\가면.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\가면.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\가문의영광.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\가문의영광.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\가위.txt" "C:\User

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\돌려차기.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\돌려차기.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\동감.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\동감.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\동갑내기_과외하기(97).txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\동갑내기_과외하기(97).hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\두사부일체.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\두사부일체.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\뚫어야산다_시나리오.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\뚫어야산다_시나리오.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\라디오스타.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\라디오스타.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\라이어.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\라이어.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\라이터를켜라.txt" "C:\Users\elina\Desktop\

hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\색즉시공.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\색즉시공.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\서프라이즈.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\서프라이즈.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\선물최종제본용87.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\선물최종제본용87.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\선생_김봉두.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\선생_김봉두.hwp"
hwp5txt --output "C:\Users\elina\Desktop\DSL\Untitled Folder\성난펭귄 19고 1011.txt" "C:\Users\elina\Desktop\DSL\Untitled Folder\성난펭귄 19고 1011.hwp"


KeyboardInterrupt: 

In [ ]:
titles

NameError: name 'titles' is not defined